In [ ]:
import cv2

In [ ]:
from deepface import DeepFace

In [ ]:
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [ ]:
def check_camera(index):
    cap = cv2.VideoCapture(index)
    if cap.isOpened():
        print(f"Camera found at index {index}")
        cap.release()
    else:
        print(f"No camera found at index {index}")

# Check both index 0 and 1
check_camera(0)
check_camera(1)

In [ ]:
cap = cv2.VideoCapture(0)

In [ ]:
while True:
    ret, frame = cap.read()
    result = DeepFace.analyze(frame, actions=['emotion'])

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray, 1.1, 4)

    # Draw a rectangle around the faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    font = cv2.FONT_HERSHEY_SIMPLEX

    # Use putText method to insert text on video
    cv2.putText(frame, 
                result['dominant_emotion'], 
                (50, 50), font, 3, (0, 0, 255), 
                2, cv2.LINE_4)

    cv2.imshow('Demo video', frame)

    if cv2.waitKey(2) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
from deepface import DeepFace
import time
import numpy as np
# Initialize webcam
cap = cv2.VideoCapture(0)

# Load Haarcascade for face detection
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

while True:
    ret, frame = cap.read()  # Read from webcam

    if not ret:
        print("Failed to capture frame")
        break  # Exit loop if frame not captured

    # Resize or process the frame
    frame = cv2.resize(frame, (320, 240))
    
    # Create a larger canvas (e.g., 480x640)
    canvas = np.zeros((480, 640, 3), dtype=np.uint8)
    
    # Place video feed in the top-left corner
    canvas[0:240, 0:320] = frame

    # Convert frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the frame
    faces = faceCascade.detectMultiScale(gray, 1.1, 4)

    # Draw rectangles around detected faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Perform emotion analysis
    try:
        result = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)
        print(result)
    except Exception as e:
        print(f"Error during face analysis: {str(e)}")
        result = None

    # Extract dominant emotion if analysis is successful
    if isinstance(result, dict) and 'dominant_emotion' in result:
        dominant_emotion = result['dominant_emotion']

        # Display detected emotion on video feed
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(frame, dominant_emotion, (50, 50), font, 1, (0, 0, 255), 2, cv2.LINE_4)

    # Show the video feed
    cv2.imshow('Demo Video', frame)

    # Check if 'q' is pressed or the window is closed
    if cv2.waitKey(1) & 0xFF == ord('q') or cv2.getWindowProperty('Demo Video', cv2.WND_PROP_VISIBLE) < 1:
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


In [1]:

import cv2
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import datetime
import random  # For demo purposes only

class DashboardApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Shift Management Dashboard")
        self.root.geometry("1200x700")  # Set a reasonable starting size
        self.root.configure(bg="#f0f0f0")
        
        # Sample data - in a real app, this would come from a database
        self.manager_name = "John Doe"
        self.employees = [
            {"id": 1, "name": "Sarah Johnson", "position": "Cashier", "performance": "High", "hours": "4/8"},
            {"id": 2, "name": "Michael Smith", "position": "Stocker", "performance": "Medium", "hours": "3/8"},
            {"id": 3, "name": "Emily Davis", "position": "Customer Service", "performance": "High", "hours": "6/8"},
            {"id": 4, "name": "Robert Wilson", "position": "Cashier", "performance": "Low", "hours": "2/8"},
            {"id": 5, "name": "Amanda Lee", "position": "Team Lead", "performance": "High", "hours": "7/8"}
        ]
        
        # Set up the layout
        self.setup_layout()
        
        # Initialize camera
        self.cap = cv2.VideoCapture(0)
        
        # Start updating elements
        self.update_time()
        self.update_video_feed()

    def setup_layout(self):
        # Create main frames with padding
        header_frame = tk.Frame(self.root, bg="#f0f0f0", pady=15, padx=20)
        header_frame.pack(fill="x")
        
        content_frame = tk.Frame(self.root, bg="#f0f0f0")
        content_frame.pack(fill="both", expand=True, padx=20, pady=10)
        
        left_frame = tk.Frame(content_frame, bg="#f0f0f0", width=700)
        left_frame.pack(side="left", fill="both", expand=True, padx=(0, 10))
        
        right_frame = tk.Frame(content_frame, bg="#f0f0f0", width=450)
        right_frame.pack(side="right", fill="both", padx=(10, 0))
        
        # HEADER SECTION
        # Welcome message (left side of header)
        welcome_frame = tk.Frame(header_frame, bg="#f0f0f0")
        welcome_frame.pack(side="left", anchor="w")
        
        welcome_label = tk.Label(welcome_frame, 
                                 text=f"Welcome, Shift Manager {self.manager_name}", 
                                 font=("Helvetica", 18, "bold"), 
                                 bg="#f0f0f0", fg="#333333")
        welcome_label.pack(anchor="w")
        
        subtitle = tk.Label(welcome_frame, 
                           text="Manage your team's performance in real-time", 
                           font=("Helvetica", 12), 
                           bg="#f0f0f0", fg="#555555")
        subtitle.pack(anchor="w")
        
        # Date and time (right side of header)
        time_frame = tk.Frame(header_frame, bg="#f0f0f0")
        time_frame.pack(side="right", anchor="e")
        
        self.date_label = tk.Label(time_frame, font=("Helvetica", 12), bg="#f0f0f0", fg="#333333")
        self.date_label.pack(anchor="e")
        
        self.time_label = tk.Label(time_frame, font=("Helvetica", 18), bg="#f0f0f0", fg="#333333")
        self.time_label.pack(anchor="e")
        
        # LEFT SECTION - Employee Information
        employee_header = tk.Label(left_frame, text="Current Shift Employees", 
                                  font=("Helvetica", 16, "bold"), bg="#f0f0f0", fg="#333333",
                                  pady=10, anchor="w")
        employee_header.pack(fill="x")
        
        # Create Treeview for employee list with scrollbar
        tree_frame = tk.Frame(left_frame)
        tree_frame.pack(fill="both", expand=True)
        
        self.tree_scroll = tk.Scrollbar(tree_frame)
        self.tree_scroll.pack(side="right", fill="y")
        
        style = ttk.Style()
        style.configure("Treeview", rowheight=40, font=("Helvetica", 12))
        style.configure("Treeview.Heading", font=("Helvetica", 12, "bold"))
        
        self.employee_tree = ttk.Treeview(tree_frame, columns=("ID", "Name", "Position", "Performance", "Hours"), 
                                         show="headings", height=10, yscrollcommand=self.tree_scroll.set)
        
        self.employee_tree.heading("ID", text="ID")
        self.employee_tree.heading("Name", text="Employee Name")
        self.employee_tree.heading("Position", text="Position")
        self.employee_tree.heading("Performance", text="Performance")
        self.employee_tree.heading("Hours", text="Hours Completed")
        
        self.employee_tree.column("ID", width=50, anchor="center")
        self.employee_tree.column("Name", width=200)
        self.employee_tree.column("Position", width=150)
        self.employee_tree.column("Performance", width=100, anchor="center")
        self.employee_tree.column("Hours", width=120, anchor="center")
        
        self.employee_tree.pack(side="left", fill="both", expand=True)
        self.tree_scroll.config(command=self.employee_tree.yview)
        
        # Populate employee data
        for employee in self.employees:
            item_id = self.employee_tree.insert("", "end", values=(
                employee["id"], 
                employee["name"], 
                employee["position"], 
                employee["performance"], 
                employee["hours"]
            ))
            
            # Color code based on performance
            if employee["performance"] == "High":
                self.employee_tree.item(item_id, tags=("high",))
            elif employee["performance"] == "Medium":
                self.employee_tree.item(item_id, tags=("medium",))
            elif employee["performance"] == "Low":
                self.employee_tree.item(item_id, tags=("low",))
        
        # Apply color tags
        self.employee_tree.tag_configure("high", background="#e6f7e6")  # Light green
        self.employee_tree.tag_configure("medium", background="#fff7e6")  # Light yellow
        self.employee_tree.tag_configure("low", background="#ffebeb")  # Light red
        
        # Summary metrics
        metrics_frame = tk.Frame(left_frame, bg="#f0f0f0", pady=15)
        metrics_frame.pack(fill="x")
        
        # Create three metric boxes
        self.create_metric_box(metrics_frame, "Total Employees", str(len(self.employees)), 0)
        self.create_metric_box(metrics_frame, "High Performers", 
                              str(sum(1 for e in self.employees if e["performance"] == "High")), 1)
        self.create_metric_box(metrics_frame, "Low Performers", 
                              str(sum(1 for e in self.employees if e["performance"] == "Low")), 2)
        
        # RIGHT SECTION - Camera feed
        camera_header = tk.Label(right_frame, text="Camera Feed - Employee Monitoring", 
                               font=("Helvetica", 16, "bold"), bg="#f0f0f0", fg="#333333",
                               pady=10, anchor="w")
        camera_header.pack(fill="x")
        
        # Create a frame for the video with a border
        video_container = tk.Frame(right_frame, bg="#ddd", bd=2, relief="groove")
        video_container.pack(fill="both", expand=True, pady=10)
        
        self.video_label = tk.Label(video_container, bg="black")
        self.video_label.pack(fill="both", expand=True, padx=2, pady=2)
        
        # Controls below video
        controls_frame = tk.Frame(right_frame, bg="#f0f0f0", pady=10)
        controls_frame.pack(fill="x")
        
        self.screenshot_btn = tk.Button(controls_frame, text="Take Screenshot", 
                                      font=("Helvetica", 12), bg="#4CAF50", fg="white",
                                      padx=10, pady=5, command=self.take_screenshot)
        self.screenshot_btn.pack(side="left", padx=5)
        
        self.settings_btn = tk.Button(controls_frame, text="Camera Settings", 
                                    font=("Helvetica", 12), bg="#2196F3", fg="white",
                                    padx=10, pady=5)
        self.settings_btn.pack(side="left", padx=5)

    def create_metric_box(self, parent, title, value, column):
        frame = tk.Frame(parent, bg="white", padx=15, pady=10, bd=1, relief="solid")
        frame.grid(row=0, column=column, padx=10, sticky="ew")
        
        parent.grid_columnconfigure(column, weight=1)
        
        title_label = tk.Label(frame, text=title, font=("Helvetica", 12), bg="white", fg="#555555")
        title_label.pack(anchor="w")
        
        value_label = tk.Label(frame, text=value, font=("Helvetica", 20, "bold"), bg="white", fg="#333333")
        value_label.pack(anchor="w")
    
    def update_time(self):
        """Update date and time labels"""
        now = datetime.datetime.now()
        date_str = now.strftime("%A, %B %d, %Y")
        time_str = now.strftime("%H:%M:%S")
        
        self.date_label.config(text=date_str)
        self.time_label.config(text=time_str)
        
        self.root.after(1000, self.update_time)
    
    def update_video_feed(self):
        """Update the video feed from the webcam"""
        ret, frame = self.cap.read()
        if ret:
            # Resize the frame to fit our layout
            frame = cv2.resize(frame, (400, 300))
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(frame)
            imgtk = ImageTk.PhotoImage(image=img)
            self.video_label.config(image=imgtk)
            self.video_label.image = imgtk
        
        self.root.after(30, self.update_video_feed)
    
    def take_screenshot(self):
        """Demo function for screenshot button"""
        print("Screenshot taken")

    def on_closing(self):
        """Handle application closing"""
        if self.cap.isOpened():
            self.cap.release()
        self.root.destroy()

# Create and run the application
if __name__ == "__main__":
    root = tk.Tk()
    app = DashboardApp(root)
    root.protocol("WM_DELETE_WINDOW", app.on_closing)
    root.mainloop()

In [10]:

# filepath: c:\Users\princ\Documents\Jan 2025\TTP\DataAnalysisPy39\RealTimeFaceEmotionRecognition.ipynb
import cv2
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import datetime
import io
from tkhtmlview import HTMLLabel, HTMLScrolledText
import base64

class DashboardApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Shift Management Dashboard")
        self.root.geometry("1200x700")
        self.root.configure(bg="#f0f0f0")
        
        # Sample data
        self.manager_name = "John Doe"
        self.employees = [
            {"id": 1, "name": "Sarah Johnson", "position": "Cashier", "performance": "High", "hours": "4/8"},
            {"id": 2, "name": "Michael Smith", "position": "Stocker", "performance": "Medium", "hours": "3/8"},
            {"id": 3, "name": "Emily Davis", "position": "Customer Service", "performance": "High", "hours": "6/8"},
            {"id": 4, "name": "Robert Wilson", "position": "Cashier", "performance": "Low", "hours": "2/8"},
            {"id": 5, "name": "Amanda Lee", "position": "Team Lead", "performance": "High", "hours": "7/8"}
        ]
        
        # Set up the layout
        self.setup_layout()
        
        # Initialize camera
        self.cap = cv2.VideoCapture(0)
        
        # Start updating elements
        self.update_time()
        self.update_video_feed()

    def setup_layout(self):
        # Create main frames with padding
        header_frame = tk.Frame(self.root, bg="#f0f0f0", pady=15, padx=20)
        header_frame.pack(fill="x")
        
        content_frame = tk.Frame(self.root, bg="#f0f0f0")
        content_frame.pack(fill="both", expand=True, padx=20, pady=10)
        
        left_frame = tk.Frame(content_frame, bg="#f0f0f0", width=700)
        left_frame.pack(side="left", fill="both", expand=True, padx=(0, 10))
        
        right_frame = tk.Frame(content_frame, bg="#f0f0f0", width=450)
        right_frame.pack(side="right", fill="both", padx=(10, 0))
        
        # HEADER SECTION using HTML
        welcome_html = f"""
        <div style="font-family: 'Segoe UI', Arial, sans-serif; padding: 10px 0;">
            <h1 style="color: #333333; font-size: 24px; margin-bottom: 5px;">Welcome, Shift Manager {self.manager_name}</h1>
            <p style="color: #555555; font-size: 16px;">Manage your team's performance in real-time</p>
        </div>
        """
        
        self.welcome_label = HTMLLabel(header_frame, html=welcome_html, background="#f0f0f0")
        self.welcome_label.pack(side="left", anchor="w", fill="x")
        
        # Date and time (right side of header)
        time_frame = tk.Frame(header_frame, bg="#f0f0f0")
        time_frame.pack(side="right", anchor="e")
        
        self.date_html = HTMLLabel(time_frame, background="#f0f0f0")
        self.date_html.pack(anchor="e")
        
        self.time_html = HTMLLabel(time_frame, background="#f0f0f0")
        self.time_html.pack(anchor="e")
        
        # LEFT SECTION - Employee Information with HTML
        employee_header_html = """
        <h2 style="color: #333333; font-size: 20px; margin-bottom: 15px; font-weight: bold;">Current Shift Employees</h2>
        """
        employee_header = HTMLLabel(left_frame, html=employee_header_html, background="#f0f0f0")
        employee_header.pack(fill="x")
        
        # Employee table HTML
        employees_html = self.generate_employees_html()
        
        # Use HTMLScrolledText for scrollable employee table
        self.employee_list = HTMLScrolledText(left_frame, html=employees_html, height=15)
        self.employee_list.pack(fill="both", expand=True, pady=10)
        
        # Metrics section with HTML
        metrics_html = self.generate_metrics_html()
        metrics_section = HTMLLabel(left_frame, html=metrics_html, background="#f0f0f0")
        metrics_section.pack(fill="x", pady=10)
        
        # RIGHT SECTION - Camera feed
        camera_header_html = """
        <h2 style="color: #333333; font-size: 20px; margin-bottom: 15px; font-weight: bold;">
            Camera Feed - Employee Monitoring
        </h2>
        """
        camera_header = HTMLLabel(right_frame, html=camera_header_html, background="#f0f0f0")
        camera_header.pack(fill="x")
        
        # Create a frame for the video with a border
        video_container = tk.Frame(right_frame, bg="#ddd", bd=2, relief="groove")
        video_container.pack(fill="both", expand=True, pady=10)
        
        self.video_label = tk.Label(video_container, bg="black")
        self.video_label.pack(fill="both", expand=True, padx=2, pady=2)
        
        # Controls below video with styled HTML buttons
        controls_html = """
        <div style="display: flex; justify-content: space-between; width: 100%; margin-top: 10px;">
            <button id="screenshot" style="
                background-color: #4CAF50;
                color: white;
                border: none;
                border-radius: 8px;
                padding: 10px 15px;
                font-size: 16px;
                cursor: pointer;
                width: 48%;
                font-weight: 500;
            ">Take Screenshot</button>
            
            <button id="settings" style="
                background-color: #2196F3;
                color: white;
                border: none;
                border-radius: 8px;
                padding: 10px 15px;
                font-size: 16px;
                cursor: pointer;
                width: 48%;
                font-weight: 500;
            ">Camera Settings</button>
        </div>
        """
        
        # Since HTML buttons won't work with Tkinter events, we'll use regular Tkinter buttons
        controls_frame = tk.Frame(right_frame, bg="#f0f0f0", pady=10)
        controls_frame.pack(fill="x")
        
        self.screenshot_btn = tk.Button(
            controls_frame, 
            text="Take Screenshot", 
            font=("Segoe UI", 12), 
            bg="#4CAF50", 
            fg="white",
            padx=10, 
            pady=5, 
            relief=tk.FLAT,
            borderwidth=0,
            command=self.take_screenshot
        )
        self.screenshot_btn.pack(side="left", padx=5, fill="x", expand=True)
        
        self.settings_btn = tk.Button(
            controls_frame, 
            text="Camera Settings", 
            font=("Segoe UI", 12), 
            bg="#2196F3", 
            fg="white",
            padx=10, 
            pady=5,
            relief=tk.FLAT,
            borderwidth=0
        )
        self.settings_btn.pack(side="right", padx=5, fill="x", expand=True)
    
    def generate_employees_html(self):
        html = """
        <style>
            table {
                width: 100%;
                border-collapse: collapse;
                font-family: 'Segoe UI', Arial, sans-serif;
            }
            th {
                background-color: #f8f9fa;
                color: #333;
                font-weight: bold;
                padding: 12px 15px;
                text-align: left;
                border-bottom: 2px solid #ddd;
            }
            td {
                padding: 12px 15px;
                border-bottom: 1px solid #ddd;
            }
            .high { background-color: #e8f5e9; }
            .medium { background-color: #fff8e1; }
            .low { background-color: #ffebee; }
        </style>
        <table>
            <thead>
                <tr>
                    <th>ID</th>
                    <th>Employee Name</th>
                    <th>Position</th>
                    <th>Performance</th>
                    <th>Hours</th>
                </tr>
            </thead>
            <tbody>
        """
        
        for employee in self.employees:
            performance_class = employee["performance"].lower()
            html += f"""
                <tr class="{performance_class}">
                    <td>{employee["id"]}</td>
                    <td>{employee["name"]}</td>
                    <td>{employee["position"]}</td>
                    <td>{employee["performance"]}</td>
                    <td>{employee["hours"]}</td>
                </tr>
            """
        
        html += """
            </tbody>
        </table>
        """
        return html

    def generate_metrics_html(self):
        high_performers = sum(1 for e in self.employees if e["performance"] == "High")
        low_performers = sum(1 for e in self.employees if e["performance"] == "Low")
        
        html = """
        <style>
            .metrics-container {{
                display: flex;
                justify-content: space-between;
                margin-top: 20px;
                font-family: 'Segoe UI', Arial, sans-serif;
            }}
            .metric-box {{
                background: white;
                padding: 20px;
                border-radius: 8px;
                width: 32%;
                box-shadow: 0 2px 5px rgba(0,0,0,0.1);
                text-align: center;
            }}
            .metric-title {{
                font-size: 14px;
                color: #555;
                margin-bottom: 10px;
            }}
            .metric-value {{
                font-size: 28px;
                font-weight: bold;
                color: #333;
            }}
        </style>
        <div class="metrics-container">
            <div class="metric-box">
                <div class="metric-title">Total Employees</div>
                <div class="metric-value">{0}</div>
            </div>
            <div class="metric-box">
                <div class="metric-title">High Performers</div>
                <div class="metric-value">{1}</div>
            </div>
            <div class="metric-box">
                <div class="metric-title">Low Performers</div>
                <div class="metric-value">{2}</div>
            </div>
        </div>
        """.format(len(self.employees), high_performers, low_performers)
        
        return html

    def update_time(self):
        """Update date and time labels with HTML"""
        now = datetime.datetime.now()
        date_str = now.strftime("%A, %B %d, %Y")
        time_str = now.strftime("%H:%M:%S")
        
        date_html = f"""
        <div style="font-size: 14px; color: #555555;">{date_str}</div>
        """
        
        time_html = f"""
        <div style="font-size: 24px; font-weight: 500; color: #333333;">{time_str}</div>
        """
        
        self.date_html.set_html(date_html)
        self.time_html.set_html(time_html)
        
        self.root.after(1000, self.update_time)
    
    def update_video_feed(self):
        """Update the video feed from the webcam"""
        try:
            ret, frame = self.cap.read()
            if ret:
                # Resize the frame to fit our layout
                frame = cv2.resize(frame, (400, 300))
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                img = Image.fromarray(frame)
                self.imgtk = ImageTk.PhotoImage(image=img)  # Store as class attribute
                self.video_label.configure(image=self.imgtk)
                self.video_label.image = self.imgtk  # Keep a reference to prevent garbage collection
            
            self.root.after(30, self.update_video_feed)
        except tk.TclError as e:
            print(f"TclError in video feed update: {e}")
            # Try to continue if possible
            self.root.after(100, self.update_video_feed)
        except Exception as e:
            print(f"Error updating video feed: {e}")
    
    def take_screenshot(self):
        """Demo function for screenshot button"""
        print("Screenshot taken")
        
        # You could add a confirmation message using HTML
        now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        message = f"""
        <div style="background-color: #e8f5e9; color: #2e7d32; padding: 10px; 
                    border-radius: 5px; margin-top: 10px; text-align: center;">
            Screenshot saved as image_{now}.jpg
        </div>
        """
        
        # Create a temporary window to show the confirmation
        confirm_window = tk.Toplevel(self.root)
        confirm_window.title("Screenshot Taken")
        confirm_window.geometry("300x100")
        
        confirm_label = HTMLLabel(confirm_window, html=message)
        confirm_label.pack(fill="both", expand=True, padx=10, pady=10)
        
        # Auto-close after 2 seconds
        confirm_window.after(2000, confirm_window.destroy)

    def on_closing(self):
        """Handle application closing"""
        if self.cap.isOpened():
            self.cap.release()
        self.root.destroy()

# Create and run the application
if __name__ == "__main__":
    root = tk.Tk()
    app = DashboardApp(root)
    root.protocol("WM_DELETE_WINDOW", app.on_closing)
    root.mainloop()


In [16]:
import cv2
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk, ImageDraw, ImageFilter
import datetime
import os
import io
import sys

class ModernDashboard:
    def __init__(self, root):
        self.root = root
        self.root.title("Shift Management Dashboard")
        self.root.geometry("1280x720")
        
        # Store the current image reference at class level
        self.current_image = None
        
        # Theme settings
        self.dark_mode = True
        
        # Colors based on theme
        self.colors = {
            "dark": {
                "bg": "#1E1E2F",
                "card": "#252A41",
                "sidebar": "#171726",
                "text": "#FFFFFF",
                "text_secondary": "#A3A6B4",
                "accent": "#6C5CE7",
                "accent_hover": "#8075E6",
                "success": "#00D084",
                "warning": "#FF9F43",
                "danger": "#FF5E5E",
                "border": "#32334D"
            },
            "light": {
                "bg": "#F5F7FB",
                "card": "#FFFFFF",
                "sidebar": "#FFFFFF",
                "text": "#2D3748",
                "text_secondary": "#718096",
                "accent": "#6C5CE7",
                "accent_hover": "#5344D7",
                "success": "#00D084",
                "warning": "#FF9F43",
                "danger": "#FF5E5E",
                "border": "#E2E8F0"
            }
        }
        
        # Fonts
        self.fonts = {
            "heading": ("Segoe UI", 18, "bold"),
            "subheading": ("Segoe UI", 16, "bold"),
            "body": ("Segoe UI", 12),
            "small": ("Segoe UI", 10),
            "body_bold": ("Segoe UI", 12, "bold"),
            "large_numbers": ("Segoe UI", 24, "bold")
        }
        
        # Sample data
        self.manager_name = "Alex Morgan"
        self.employees = [
            {"id": 1, "name": "Sarah Johnson", "position": "Cashier", "performance": "High", "hours": "4/8"},
            {"id": 2, "name": "Michael Smith", "position": "Stocker", "performance": "Medium", "hours": "3/8"},
            {"id": 3, "name": "Emily Davis", "position": "Customer Service", "performance": "High", "hours": "6/8"},
            {"id": 4, "name": "Robert Wilson", "position": "Cashier", "performance": "Low", "hours": "2/8"},
            {"id": 5, "name": "Amanda Lee", "position": "Team Lead", "performance": "High", "hours": "7/8"}
        ]
        
        # Set up the layout
        self.create_base_layout()
        
        # Initialize camera
        self.cap = cv2.VideoCapture(0)
        
        # Start updating elements
        self.update_time()
        self.update_video_feed()
        
    def get_color(self, name):
        """Get color based on current theme"""
        theme = "dark" if self.dark_mode else "light"
        return self.colors[theme][name]
    
    def create_rounded_rectangle(self, width, height, radius=20, fill="#252A41"):
        """Create a rounded rectangle image for backgrounds"""
        rect_img = Image.new('RGBA', (width, height), (0, 0, 0, 0))
        draw = ImageDraw.Draw(rect_img)
        
        # Draw rounded rectangle
        draw.rounded_rectangle([(0, 0), (width, height)], radius, fill=fill)
        
        return rect_img
    
    def create_card_bg(self, width, height, radius=20):
        """Create a card background with rounded corners and subtle shadow"""
        # Create base rounded rectangle
        card = self.create_rounded_rectangle(width, height, radius, self.get_color("card"))
        
        # Convert to PhotoImage
        return ImageTk.PhotoImage(card)
    
    def toggle_theme(self):
        """Toggle between dark and light mode"""
        self.dark_mode = not self.dark_mode
        self.refresh_theme()
        
    def refresh_theme(self):
        """Update all UI elements to match current theme"""
        # Update main background
        self.root.configure(bg=self.get_color("bg"))
        
        # Update sidebar
        self.sidebar_frame.configure(bg=self.get_color("sidebar"))
        self.sidebar_header.configure(bg=self.get_color("sidebar"), fg=self.get_color("text"))
        
        for btn in self.sidebar_buttons:
            btn.configure(
                bg=self.get_color("sidebar"),
                fg=self.get_color("text_secondary"),
                activebackground=self.get_color("accent"),
                activeforeground=self.get_color("text")
            )
            
        # Update theme toggle button
        icon = "☀️" if self.dark_mode else "🌙"
        self.theme_btn.configure(text=icon)
        
        # Update content area
        self.content_frame.configure(bg=self.get_color("bg"))
        
        # Update content cards (recreate them)
        for widget in self.main_content.winfo_children():
            widget.destroy()
            
        self.create_dashboard_content()
    
    def create_base_layout(self):
        """Create the basic layout with sidebar and content area"""
        # Configure root
        self.root.configure(bg=self.get_color("bg"))
        
        # Create main container as a grid with 2 columns
        self.main_container = tk.Frame(self.root, bg=self.get_color("bg"))
        self.main_container.pack(fill="both", expand=True)
        
        # 1. Create sidebar (left column)
        sidebar_width = 240
        self.sidebar_frame = tk.Frame(
            self.main_container, 
            width=sidebar_width, 
            bg=self.get_color("sidebar"),
            padx=20,
            pady=20
        )
        self.sidebar_frame.pack(side="left", fill="y")
        self.sidebar_frame.pack_propagate(False)  # Don't shrink
        
        # Sidebar header with app name
        self.sidebar_header = tk.Label(
            self.sidebar_frame, 
            text="SHIFT MANAGER", 
            font=self.fonts["heading"],
            bg=self.get_color("sidebar"),
            fg=self.get_color("text"),
            anchor="w",
            pady=15
        )
        self.sidebar_header.pack(fill="x")
        
        # Create navigation buttons with hover effect
        self.sidebar_buttons = []
        
        nav_items = [
            ("📊 Dashboard", self.show_dashboard),
            ("👥 Employees", self.show_dashboard),
            ("📈 Analytics", self.show_dashboard),
            ("⚙️ Settings", self.show_dashboard),
        ]
        
        for text, command in nav_items:
            btn = tk.Button(
                self.sidebar_frame,
                text=text,
                font=self.fonts["body"],
                bg=self.get_color("sidebar"),
                fg=self.get_color("text_secondary"),
                activebackground=self.get_color("accent"),
                activeforeground=self.get_color("text"),
                bd=0,
                cursor="hand2",
                anchor="w",
                padx=10,
                pady=12,
                command=command,
                relief="flat",
                highlightthickness=0
            )
            btn.pack(fill="x", pady=3)
            self.sidebar_buttons.append(btn)
            
            # Bind hover events
            btn.bind("<Enter>", lambda e, b=btn: b.configure(
                bg=self.get_color("accent"), 
                fg=self.get_color("text")
            ))
            btn.bind("<Leave>", lambda e, b=btn: b.configure(
                bg=self.get_color("sidebar"), 
                fg=self.get_color("text_secondary")
            ))
            
        # Add theme toggle at the bottom
        theme_frame = tk.Frame(
            self.sidebar_frame, 
            bg=self.get_color("sidebar"),
            pady=20
        )
        theme_frame.pack(side="bottom", fill="x")
        
        self.theme_btn = tk.Button(
            theme_frame,
            text="☀️",  # Sun icon for dark mode, will change to moon in light mode
            font=("Segoe UI", 16),
            bg=self.get_color("sidebar"),
            fg=self.get_color("text"),
            bd=0,
            cursor="hand2",
            command=self.toggle_theme,
            activebackground=self.get_color("sidebar")
        )
        self.theme_btn.pack(side="right")
        
        # 2. Create content area (right column)
        self.content_frame = tk.Frame(
            self.main_container, 
            bg=self.get_color("bg"),
            padx=30,
            pady=30
        )
        self.content_frame.pack(side="right", fill="both", expand=True)
        
        # Header with welcome message and date/time
        self.header_frame = tk.Frame(self.content_frame, bg=self.get_color("bg"))
        self.header_frame.pack(fill="x", pady=(0, 20))
        
        # Welcome message
        welcome_frame = tk.Frame(self.header_frame, bg=self.get_color("bg"))
        welcome_frame.pack(side="left", anchor="w")
        
        self.welcome_text = tk.Label(
            welcome_frame,
            text=f"Welcome back, {self.manager_name}",
            font=self.fonts["heading"],
            bg=self.get_color("bg"),
            fg=self.get_color("text")
        )
        self.welcome_text.pack(anchor="w")
        
        self.welcome_subtext = tk.Label(
            welcome_frame,
            text="Here's what's happening with your team today",
            font=self.fonts["body"],
            bg=self.get_color("bg"),
            fg=self.get_color("text_secondary")
        )
        self.welcome_subtext.pack(anchor="w")
        
        # Date and time
        time_frame = tk.Frame(self.header_frame, bg=self.get_color("bg"))
        time_frame.pack(side="right", anchor="e")
        
        self.date_label = tk.Label(
            time_frame,
            font=self.fonts["body"],
            bg=self.get_color("bg"),
            fg=self.get_color("text_secondary")
        )
        self.date_label.pack(anchor="e")
        
        self.time_label = tk.Label(
            time_frame,
            font=self.fonts["heading"],
            bg=self.get_color("bg"),
            fg=self.get_color("text")
        )
        self.time_label.pack(anchor="e")
        
        # Main content area - will be populated by specific views
        self.main_content = tk.Frame(self.content_frame, bg=self.get_color("bg"))
        self.main_content.pack(fill="both", expand=True)
        
        # Show default dashboard view
        self.create_dashboard_content()
    
    def create_dashboard_content(self):
        """Create the dashboard content with metrics, employee list and camera feed"""
        # Create the main grid layout: 2x2 grid
        self.main_content.columnconfigure(0, weight=3)
        self.main_content.columnconfigure(1, weight=2)
        self.main_content.rowconfigure(0, weight=1)
        self.main_content.rowconfigure(1, weight=1)
        
        # 1. Top left: KPI metrics
        kpi_frame = self.create_card_frame("Team Performance")
        kpi_frame.grid(row=0, column=0, padx=(0, 10), pady=(0, 10), sticky="nsew")
        
        # Create metric boxes inside
        metric_grid = tk.Frame(kpi_frame, bg=self.get_color("card"))
        metric_grid.pack(fill="both", expand=True, padx=15, pady=15)
        
        metric_grid.columnconfigure(0, weight=1)
        metric_grid.columnconfigure(1, weight=1)
        metric_grid.columnconfigure(2, weight=1)
        
        metrics = [
            {"title": "Total Employees", "value": str(len(self.employees)), "icon": "👥"},
            {"title": "High Performers", "value": str(sum(1 for e in self.employees if e["performance"] == "High")), "icon": "🌟"},
            {"title": "Low Performers", "value": str(sum(1 for e in self.employees if e["performance"] == "Low")), "icon": "⚠️"}
        ]
        
        for i, metric in enumerate(metrics):
            self.create_metric_box(metric_grid, metric["title"], metric["value"], metric["icon"], i)
            
        # 2. Top right: Camera feed
        camera_frame = self.create_card_frame("Employee Monitoring")
        camera_frame.grid(row=0, column=1, padx=(10, 0), pady=(0, 10), sticky="nsew")
        
        self.camera_container = tk.Frame(camera_frame, bg=self.get_color("card"))
        self.camera_container.pack(fill="both", expand=True, padx=15, pady=15)
        
        self.video_label = tk.Label(self.camera_container, bg="black")
        self.video_label.pack(fill="both", expand=True)
        
        # Camera controls
        controls_frame = tk.Frame(self.camera_container, bg=self.get_color("card"), pady=10)
        controls_frame.pack(fill="x")
        
        self.screenshot_btn = self.create_button(
            controls_frame, "📷 Take Screenshot", self.take_screenshot,
            bg=self.get_color("accent")
        )
        self.screenshot_btn.pack(side="left", padx=(0, 5), fill="x", expand=True)
        
        self.settings_btn = self.create_button(
            controls_frame, "⚙️ Camera Settings", lambda: None,
            bg=self.get_color("text_secondary")
        )
        self.settings_btn.pack(side="right", padx=(5, 0), fill="x", expand=True)
        
        # 3. Bottom: Employee list
        employee_frame = self.create_card_frame("Current Shift Employees")
        employee_frame.grid(row=1, column=0, columnspan=2, sticky="nsew", pady=(10, 0))
        
        tree_container = tk.Frame(employee_frame, bg=self.get_color("card"), padx=15, pady=15)
        tree_container.pack(fill="both", expand=True)
        
        # Configure treeview style for the current theme
        style = ttk.Style()
        style.theme_use("clam")  # Use the clam theme as base
        
        # Configure the Treeview widget colors according to our theme
        style.configure(
            "Custom.Treeview",
            background=self.get_color("card"),
            foreground=self.get_color("text"),
            fieldbackground=self.get_color("card"),
            rowheight=40
        )
        style.configure(
            "Custom.Treeview.Heading",
            background=self.get_color("bg"),
            foreground=self.get_color("text"),
            font=self.fonts["body_bold"]
        )
        
        # Set up the tree view with scrollbar
        tree_frame = tk.Frame(tree_container, bg=self.get_color("card"))
        tree_frame.pack(fill="both", expand=True)
        
        self.tree_scroll = tk.Scrollbar(tree_frame, bg=self.get_color("card"))
        self.tree_scroll.pack(side="right", fill="y")
        
        self.employee_tree = ttk.Treeview(
            tree_frame,
            columns=("ID", "Name", "Position", "Performance", "Hours"),
            show="headings",
            style="Custom.Treeview",
            height=5,
            yscrollcommand=self.tree_scroll.set
        )
        
        # Define headings
        self.employee_tree.heading("ID", text="ID")
        self.employee_tree.heading("Name", text="Employee Name")
        self.employee_tree.heading("Position", text="Position")
        self.employee_tree.heading("Performance", text="Performance")
        self.employee_tree.heading("Hours", text="Hours Completed")
        
        # Define columns
        self.employee_tree.column("ID", width=50, anchor="center")
        self.employee_tree.column("Name", width=200)
        self.employee_tree.column("Position", width=150)
        self.employee_tree.column("Performance", width=100, anchor="center")
        self.employee_tree.column("Hours", width=120, anchor="center")
        
        self.employee_tree.pack(side="left", fill="both", expand=True)
        self.tree_scroll.config(command=self.employee_tree.yview)
        
        # Populate employee data
        for employee in self.employees:
            item_id = self.employee_tree.insert("", "end", values=(
                employee["id"], 
                employee["name"], 
                employee["position"], 
                employee["performance"], 
                employee["hours"]
            ))
            
            # Color code based on performance
            if employee["performance"] == "High":
                self.employee_tree.item(item_id, tags=("high",))
            elif employee["performance"] == "Medium":
                self.employee_tree.item(item_id, tags=("medium",))
            elif employee["performance"] == "Low":
                self.employee_tree.item(item_id, tags=("low",))
        
        # Apply color tags - using lighter versions of colors instead of alpha transparency
        def lighten_color(color_hex):
            # Convert hex to RGB and make it lighter
            r = int(color_hex[1:3], 16)
            g = int(color_hex[3:5], 16)
            b = int(color_hex[5:7], 16)
            # Blend with white to create a lighter version (simulating transparency)
            r = r + int((255 - r) * 0.8)  # 80% lighter
            g = g + int((255 - g) * 0.8)
            b = b + int((255 - b) * 0.8)
            return f"#{r:02x}{g:02x}{b:02x}"
            
        self.employee_tree.tag_configure("high", background=lighten_color(self.get_color("success")))
        self.employee_tree.tag_configure("medium", background=lighten_color(self.get_color("warning")))
        self.employee_tree.tag_configure("low", background=lighten_color(self.get_color("danger")))
    
    def create_card_frame(self, title):
        """Create a card-like frame with title"""
        card_frame = tk.Frame(
            self.main_content,
            bg=self.get_color("card"),
            highlightbackground=self.get_color("border"),
            highlightthickness=1,
            bd=0
        )
        
        # Round corners using the highlightbackground trick
        # (not perfect but best we can do without custom drawing)
        
        # Title
        title_label = tk.Label(
            card_frame,
            text=title,
            font=self.fonts["subheading"],
            bg=self.get_color("card"),
            fg=self.get_color("text"),
            anchor="w",
            pady=15,
            padx=15
        )
        title_label.pack(fill="x")
        
        # Add separator
        separator = tk.Frame(
            card_frame,
            height=1,
            bg=self.get_color("border")
        )
        separator.pack(fill="x")
        
        return card_frame
    
    def create_button(self, parent, text, command, bg=None):
        if bg is None:
            bg = self.get_color("accent")
            
        btn = tk.Button(
            parent,
            text=text,
            font=self.fonts["body"],
            bg=bg,
            fg=self.get_color("text"),
            activebackground=self.get_color("accent_hover"),
            activeforeground=self.get_color("text"),
            bd=0,
            cursor="hand2",
            padx=15,
            pady=8,
            command=command,
            relief="flat"
        )
        
        # Add hover effect
        btn.bind("<Enter>", lambda e, b=btn: b.configure(bg=self.get_color("accent_hover")))
        btn.bind("<Leave>", lambda e, b=btn, orig_bg=bg: b.configure(bg=orig_bg))
        
        return btn
    
    def create_metric_box(self, parent, title, value, icon, col):
        metric_frame = tk.Frame(
            parent,
            bg=self.get_color("card"),
            padx=15,
            pady=15
        )
        metric_frame.grid(row=0, column=col, sticky="nsew", padx=5)
        
        # Icon and title in same row
        header_frame = tk.Frame(metric_frame, bg=self.get_color("card"))
        header_frame.pack(fill="x", anchor="w")
        
        # Icon (emoji)
        icon_label = tk.Label(
            header_frame,
            text=icon,
            font=("Segoe UI", 20),
            bg=self.get_color("card"),
            fg=self.get_color("text")
        )
        icon_label.pack(side="left", anchor="w")
        
        title_label = tk.Label(
            header_frame,
            text=title,
            font=self.fonts["body"],
            bg=self.get_color("card"),
            fg=self.get_color("text_secondary"),
            padx=10
        )
        title_label.pack(side="left", anchor="w")
        
        # Value
        value_label = tk.Label(
            metric_frame,
            text=value,
            font=self.fonts["large_numbers"],
            bg=self.get_color("card"),
            fg=self.get_color("text"),
            pady=10
        )
        value_label.pack(anchor="w")
    
    def update_time(self):
        """Update date and time labels"""
        now = datetime.datetime.now()
        date_str = now.strftime("%A, %B %d, %Y")
        time_str = now.strftime("%H:%M:%S")
        
        self.date_label.config(text=date_str)
        self.time_label.config(text=time_str)
    def update_video_feed(self):
        """Update the video feed from the webcam"""
        try:
            if self.cap.isOpened():
                ret, frame = self.cap.read()
                if ret:
                    # Resize the frame to fit our layout
                    frame = cv2.resize(frame, (400, 300))
                    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    img = Image.fromarray(frame)
                    self.current_image = ImageTk.PhotoImage(image=img)
                    self.video_label.config(image=self.current_image)
                    # Keep a reference to prevent garbage collection
                    self.video_label.image = self.current_image
                else:
                    # Handle camera read failure
                    print("Failed to read frame from camera")
            else:
                # Try to reinitialize the camera if it's closed
                print("Camera not open, attempting to reopen...")
                self.cap = cv2.VideoCapture(0)
        except Exception as e:
            print(f"Error in video feed: {e}")
        
        # Schedule the next update
        self.root.after(30, self.update_video_feed)
    
    def take_screenshot(self):
        """Demo function for screenshot button"""
        print("Screenshot taken")

    def show_dashboard(self):
        """Show the dashboard view (already implemented)"""
        pass  # We're already showing it
    
    def on_closing(self):
        """Handle application closing"""
        if self.cap.isOpened():
            self.cap.release()
        self.root.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = ModernDashboard(root)
    root.protocol("WM_DELETE_WINDOW", app.on_closing)
    root.mainloop()

KeyboardInterrupt: 